In [256]:
from openpyxl import Workbook, load_workbook
import pandas as pd
from openpyxl.utils.dataframe import dataframe_to_rows as df_to_row
import numpy as np
from math import ceil

In [3]:
path_to_data = 'C:/Users/PetukhovMD/Desktop/szp/'
path_to_docs = 'C:/Users/PetukhovMD/Desktop/справочники/'
months = ['feb.xlsx', 'jan.xlsx']
sheet = 'Sheet'

In [4]:
def to_double(row, f_name):
    if type(row[f_name]) is float:
        return row[f_name] 
    t = str(row[f_name]).split(',')
    if t[0] == 'nan':
        return 0.0
    return float(t[0]+'.'+t[1])

In [282]:
df = pd.read_excel(path_to_data + 'jan.xlsx')
df.stv = df.apply(lambda row: to_double(row, 'stv'), axis=1)
sample = df[(df.stv >= 1.0) & (df.status == 'Работа') & (df.day == 1) & ((df.type == 'Основное место работы') | (df.type == 'Внутреннее совместительство'))]
sums = sample.iloc[:, [0,1,2,3]].groupby(['inn', 'snils']).sum().reset_index()
sums = pd.merge(sums, sample[sample.type == 'Основное место работы'], how='left', on='snils')
sums = sums[['inn_x', 'snils', 'sum_x', 'job']].drop_duplicates(['inn_x', 'snils', 'sum_x'])
sums = sums.rename(columns = {'inn_x': 'inn', 'sum_x': 'sum_jan', 'job':'job_jan'})
sums


,inn,snils,sum_jan,job_jan
0,5003021368,001-191-281 85,106626.88,Учитель
1,5003021368,001-244-107 65,109394.87,Учитель
2,5003021368,001-263-234 77,74520.00,Воспитатель
3,5003021368,001-322-594 80,83013.52,Специалист
4,5003021368,001-519-663 10,142589.38,Учитель
...,...,...,...,...
113367,9718071371,196-882-391 45,77596.00,Бухгалтер
113368,9718071371,198-036-430 93,110851.00,Бухгалтер
113369,9718071371,199-418-719 35,131067.00,Бухгалтер
113370,9718071371,205-412-630 13,189077.50,Начальник группы


In [284]:
df = pd.read_excel(path_to_data + 'feb.xlsx')
df.stv = df.apply(lambda row: to_double(row, 'stv'), axis=1)
sample = df[(df.stv >= 1.0) & (df.status == 'Работа') & (df.day == 1) & ((df.type == 'Основное место работы') | (df.type == 'Внутреннее совместительство'))]
sums = sample.iloc[:, [0,1,2,3]].groupby(['inn', 'snils']).sum().reset_index()
sums = pd.merge(sums, sample[sample.type == 'Основное место работы'], how='left', on='snils')
sums = sums[['inn_x', 'snils', 'sum_x', 'job']].drop_duplicates(['inn_x', 'snils', 'sum_x'])
sums = sums.rename(columns = {'inn_x': 'inn', 'sum_x': 'sum_feb', 'job':'job_feb'})

In [283]:
res = sums

In [286]:
res

,inn,snils,sum_jan,job_jan,sum_feb,job_feb
0,5003021368,001-191-281 85,106626.88,Учитель,139753.92,Учитель
1,5003021368,001-244-107 65,109394.87,Учитель,109541.13,Учитель
2,5003021368,001-263-234 77,74520.00,Воспитатель,81178.00,Воспитатель
3,5003021368,001-322-594 80,83013.52,Специалист,75815.00,Специалист
4,5003021368,001-519-663 10,142589.38,Учитель,163810.32,Учитель
...,...,...,...,...,...,...
131871,9718071371,166-448-762 08,NaN,NaN,110851.00,Бухгалтер
131872,9718071371,173-386-417 88,NaN,NaN,101079.63,Бухгалтер
131873,9718071371,175-876-580 36,NaN,NaN,110851.00,Бухгалтер
131874,9718071371,176-772-921 22,NaN,NaN,110851.00,Инженер-сметчик


In [285]:
res = res.merge(sums, how= 'outer', on=['inn', 'snils'])

In [141]:
def print_df(df, name):
    wb = Workbook()
    ws = wb.active
    for  i in df_to_row(df, header=True):
        ws.append(i)
    wb.save(name + '.xlsx')

In [142]:
print_df(res[(res.inn == 5003021368) & (res.job_feb == 'Учитель')], 'teach_check')

In [287]:
res[(res.inn == 9718071371)]

,inn,snils,sum_jan,job_jan,sum_feb,job_feb
112515,9718071371,001-102-623 46,147930.0,ведущий Бухгалтер,157930.00,ведущий Бухгалтер
112516,9718071371,001-184-402 83,177930.0,ведущий Бухгалтер,177930.00,ведущий Бухгалтер
112517,9718071371,001-200-216 33,115851.0,Бухгалтер,142896.32,Бухгалтер
112518,9718071371,001-208-598 92,96078.0,Бухгалтер,117705.00,Бухгалтер
112519,9718071371,001-257-058 90,114851.0,Бухгалтер,110851.00,Бухгалтер
...,...,...,...,...,...,...
131871,9718071371,166-448-762 08,NaN,NaN,110851.00,Бухгалтер
131872,9718071371,173-386-417 88,NaN,NaN,101079.63,Бухгалтер
131873,9718071371,175-876-580 36,NaN,NaN,110851.00,Бухгалтер
131874,9718071371,176-772-921 22,NaN,NaN,110851.00,Инженер-сметчик


In [48]:
def szp(row, months):
    if pd.isna(row['job_' + months[-1]]):
        return np.nan
    cnt = 0
    sum_szp = 0
    for month in months:
        if pd.isna(row['sum_' + month]):
            continue
        cnt += 1
        sum_szp += row['sum_' + month]
    # if row['inn'] == 9715217689 and row['snils'] == '173-260-871 66':
    #     print(cnt, sum_szp)
    return sum_szp / cnt

In [288]:
res['szp'] = res.apply(lambda row: szp(row, ['jan', 'feb']), axis= 1)

In [55]:
depr_month_inc = 84100.0

In [20]:
df = pd.read_excel(path_to_docs + 'input.xlsx', sheet_name='УГД')

In [24]:
df

,Должность,УГД,Педагогический,Работники непосредственно осуществляющие и обеспечивающие основной учебно-вспомогательный процесс во взаимодействии с детьми
0,Главный бухгалтер,Административно-управленческий персонал,0,0
1,Директор,Административно-управленческий персонал,0,0
2,Директор филиала,Административно-управленческий персонал,0,0
3,Заведующий,Административно-управленческий персонал,0,0
4,Заместитель главного бухгалтера,Административно-управленческий персонал,0,0
...,...,...,...,...
291,Эксперт,Прочий персонал (включая иной административный...,0,0
292,Электромеханик,Прочий персонал (включая иной административный...,0,0
293,Старший юрисконсульт,Прочий персонал (включая иной административный...,0,0
294,Ведущий юрисконсульт,Прочий персонал (включая иной административный...,0,0


In [23]:
ped = df[df['Педагогический'] == 1]['Должность'].tolist()
ped

['Ассистент',
 'Декан факультета',
 'Директор института',
 'Доцент',
 'Заведующий кафедрой',
 'Профессор',
 'Преподаватель',
 'Старший преподаватель',
 'Воспитатель',
 'Инструктор-методист',
 'Инструктор по труду',
 'Инструктор по физической культуре',
 'Концертмейстер',
 'Мастер производственного обучения',
 'Методист',
 'Музыкальный руководитель',
 'Педагог дополнительного образования',
 'Педагог-библиотекарь',
 'Педагог-организатор',
 'Педагог-психолог ',
 'Преподаватель ',
 'Преподаватель-организатор основ безопасности жизнедеятельности',
 'Руководитель физического воспитания',
 'Социальный педагог',
 'Старший вожатый',
 'Старший воспитатель',
 'Старший инструктор-методист',
 'Старший методист',
 'Старший педагог дополнительного образования',
 'Старший педагог-психолог',
 'Старший тренер-преподаватель',
 'Тренер-преподаватель',
 'Тьютор',
 'Учитель',
 'Учитель-дефектолог',
 'Учитель-логопед']

In [25]:
isp = df[df['Работники непосредственно осуществляющие и обеспечивающие основной учебно-вспомогательный процесс во взаимодействии с детьми'] == 1]['Должность'].tolist()
isp

['Преподаватель',
 'Воспитатель',
 'Инструктор по физической культуре',
 'Концертмейстер',
 'Мастер производственного обучения',
 'Музыкальный руководитель',
 'Педагог дополнительного образования',
 'Педагог-организатор',
 'Педагог-психолог ',
 'Преподаватель ',
 'Социальный педагог',
 'Тьютор',
 'Учитель',
 'Учитель-дефектолог',
 'Учитель-логопед',
 'Помощник воспитателя',
 'Младший воспитатель']

In [51]:
def szp_ped(row, months):
    sum_szp = 0
    cnt = 0
    for month in months:
        if row['job_' + month] in ped:
            cnt += 1
            sum_szp += row['sum_' + month]
    if cnt == 0 or pd.isna(row['job_' + months[-1]]):
        return np.nan
    return sum_szp / cnt

In [224]:
def szp_teach(row, months):
    sum_szp = 0
    cnt = 0
    for month in months:
        if row['job_' + month] in ['Учитель']:
            cnt += 1
            sum_szp += row['sum_' + month]
    if cnt == 0 or row['job_' + months[-1]] != 'Учитель':
        return np.nan
    return sum_szp / cnt

In [289]:
res['szp_ped'] = res.apply(lambda row: szp_ped(row, ['jan', 'feb']), axis=1)

In [290]:
res['szp_teach'] = res.apply(lambda row: szp_teach(row, ['jan', 'feb']), axis=1)

In [291]:
val_5 = res[['inn', 'szp_ped']].groupby('inn').mean()
val_5['value'] = val_5['szp_ped'] / depr_month_inc
val_5

,szp_ped,value
inn,,
5003021368,102251.195455,1.215829
5003021495,109061.435581,1.296807
5003021640,117003.647170,1.391244
5003096290,100078.023906,1.189988
5030032182,105850.473651,1.258626
...,...,...
7743940590,136533.929551,1.623471
9705101759,NaN,NaN
9715206976,106223.829945,1.263066


In [292]:
print_df(val_5, 'val_5')
print_df(res[res.szp_ped.isna() == False][['inn', 'snils', 'szp_ped']], 'val_5_ppl')

In [293]:
val_6 = res[['inn', 'szp_ped']].groupby('inn').quantile(0.5).rename(columns={'szp_ped': 'q'})
val_6 = pd.merge(val_6, res[['inn', 'snils', 'szp_ped']], how='right', on='inn')
val_6_out = val_6
val_6 = val_6[val_6.szp_ped <= val_6.q][['inn', 'szp_ped']].groupby('inn').mean()
val_6

,szp_ped
inn,
5003021368,75640.146839
5003021495,85421.727149
5003021640,86247.207554
5003096290,71240.875243
5030032182,76387.475952
...,...
7743937357,98423.758214
7743939315,80638.806429
7743940590,111136.985385


In [294]:
print_df(val_6, 'val_6')
print_df(val_6_out, 'val_6_ppl')

In [227]:
res[(res.job_jan == 'Учитель') & (res.szp_teach == np.nan)]

,inn,snils,sum_jan,job_jan,sum_feb,job_feb,szp,szp_ped,szp_teach


In [295]:
val_7 = pd.merge(res[['inn', 'szp_teach']].groupby('inn').count().rename(columns={'szp_teach': 'cnt1'}), 
res[res.szp_teach >= 84100][['inn', 'szp_teach']].groupby('inn').count().rename(columns={'szp_teach': 'cnt2'}), how='outer', on='inn')
val_7['val'] = val_7.cnt2 / val_7.cnt1
val_7[val_7.val < 1]

,cnt1,cnt2,val
inn,,,
5003021368,209,208.0,0.995215
5030032182,47,46.0,0.978723
5051005623,41,40.0,0.975610
5074045703,135,132.0,0.977778
7701905508,59,58.0,0.983051
...,...,...,...
7743868110,53,49.0,0.924528
7743917311,70,69.0,0.985714
7743928539,17,16.0,0.941176


In [296]:
print_df(val_7, 'val_7')
print_df(res[res.szp_teach.isna() == False][['inn', 'snils', 'szp_teach']], 'val_7_ppl')

In [206]:
def q_cnt1(row):
    if pd.isna(row['szp']):
        return np.nan
    return row['szp'] if row['szp'] <= row['0.1'] else np.nan

In [207]:
def q_cnt2(row):
    if pd.isna(row['szp']):
        return np.nan
    return row['szp'] if row['szp'] >= row['0.9'] else np.nan

In [297]:
q01 = res[['inn','szp']].groupby('inn').quantile(0.1).rename(columns={'szp':'0.1'})
q09 = res[['inn','szp']].groupby('inn').quantile(0.9).rename(columns={'szp':'0.9'})
q = pd.merge(q01, q09, how='right', on='inn')
qm = pd.merge(res[['inn', 'szp']], q, how='left', on='inn')
qm['cnt1'] = qm.apply(lambda row: q_cnt1(row), axis=1)
qm['cnt2'] = qm.apply(lambda row: q_cnt2(row), axis=1)
qm_pers = pd.merge(qm[qm['szp'].isna() == False], res[['inn','szp', 'snils']], how='left', on=['inn', 'szp']).drop_duplicates(['inn', 'snils'])
qm = qm[['inn', 'cnt1', 'cnt2']].groupby('inn').mean()
qm['ans'] = qm.cnt2 / qm.cnt1
qm[qm.ans > 3.7]

,cnt1,cnt2,ans
inn,,,
7701293580,74268.750000,287666.510625,3.873318
7702061938,75662.750000,345546.666667,4.566932
7703738796,53931.551429,205599.179286,3.812224
7704040242,85182.432222,388053.534444,4.555558
7704118139,55315.714286,237911.405000,4.300973
...,...,...,...
7743023522,51797.185233,197739.932326,3.817581
7743024406,52700.617500,197218.967143,3.742252
7743085712,78738.851333,370557.365333,4.706157


In [197]:
q01[q01['0.1'].isna()]

,0.1
inn,


In [298]:
print_df(qm, 'dezil')
print_df(qm_pers, 'dezil_pers')

In [299]:
cut_off = res[['inn', 'szp']].groupby('inn').count().rename(columns={'szp': 'cnt'})
cut_off.cnt = np.ceil(cut_off.cnt / 10)
cut_off = cut_off.to_dict()['cnt']
cut_off

{5003021368: 61.0,
 5003021495: 58.0,
 5003021640: 51.0,
 5003096290: 38.0,
 5030032182: 17.0,
 5030032376: 35.0,
 5051005503: 22.0,
 5051005623: 15.0,
 5051005670: 68.0,
 5074019220: 20.0,
 5074019252: 33.0,
 5074045703: 38.0,
 7701032028: 3.0,
 7701050524: 12.0,
 7701113894: 5.0,
 7701293580: 8.0,
 7701375995: 5.0,
 7701377657: 15.0,
 7701384943: 3.0,
 7701406844: 5.0,
 7701905508: 20.0,
 7702058364: 12.0,
 7702061938: 3.0,
 7702764296: 11.0,
 7702797527: 20.0,
 7702833359: 20.0,
 7703021039: 6.0,
 7703112310: 6.0,
 7703261626: 10.0,
 7703363890: 18.0,
 7703379868: 3.0,
 7703611221: 13.0,
 7703738796: 14.0,
 7703742922: 27.0,
 7703776689: 8.0,
 7703808740: 8.0,
 7704040242: 9.0,
 7704118121: 5.0,
 7704118139: 7.0,
 7704178709: 12.0,
 7704191153: 7.0,
 7704222919: 13.0,
 7704222926: 28.0,
 7704231631: 23.0,
 7704234760: 19.0,
 7704271031: 1.0,
 7704853583: 14.0,
 7705020295: 13.0,
 7705041520: 3.0,
 7705052410: 6.0,
 7705399348: 23.0,
 7705480398: 22.0,
 7705513678: 21.0,
 7705513734:

In [300]:
ppl = res[['inn', 'szp']].sort_values('szp').dropna().groupby('inn')
keys = ppl.groups.keys()
val_1_7 = pd.DataFrame({'inn': [], 'val': []})
val_1_7

,inn,val


In [301]:
for key in keys:
    top = ppl.get_group(key).tail(int(cut_off[key]))['szp'].mean()
    bot = ppl.get_group(key).head(int(cut_off[key]))['szp'].mean()
    t_df = pd.DataFrame({'inn': [key], 'val': [top / bot]})
    val_1_7 = pd.concat([val_1_7, t_df])

In [302]:
print_df(val_1_7, 'val_1_7')

In [303]:
cnt1 = res[res.job_feb == 'Учитель'][['inn', 'szp']].groupby('inn').mean().rename(columns={'szp':'cnt1'})
cnt2 = res[res.job_feb == 'Учитель'][['inn', 'szp']].groupby('inn').quantile(0.5).rename(columns={'szp':'q'})
cnt2 = pd.merge(cnt2, res[res.job_feb == 'Учитель'][['inn', 'snils', 'szp']], how='right', on='inn')
teach_perc_out = cnt2
cnt2 = cnt2[cnt2.szp <= cnt2.q][['inn', 'szp']].groupby('inn').mean().rename(columns={'szp':'cnt2'})
val_15 = pd.merge(cnt2, cnt1, how='left', on='inn')
val_15['val'] = val_15.cnt1 / val_15.cnt2
val_15[val_15.val >= 1.4]

,cnt2,cnt1,val
inn,,,


In [304]:
print_df(val_15, 'val_15')
print_df(teach_perc_out, 'val_15_ppl')

In [239]:
def summ_isp(row, months):
    sum = 0
    for month in months:
        if row['job_' + month] in isp:
            sum += row['sum_' + month]
    if pd.isna(row['job_' + months[-1]]):
        return np.nan
    return sum

In [249]:
res

,inn,snils,sum_jan,job_jan,sum_feb,job_feb,szp,szp_ped,szp_teach,summ
0,5003021368,001-191-281 85,106626.88,Учитель,139753.92,Учитель,123190.40,123190.40,123190.40,246380.80
1,5003021368,001-244-107 65,109394.87,Учитель,109541.13,Учитель,109468.00,109468.00,109468.00,218936.00
2,5003021368,001-263-234 77,74520.00,Воспитатель,81178.00,Воспитатель,77849.00,77849.00,NaN,155698.00
3,5003021368,001-322-594 80,83013.52,Специалист,75815.00,Специалист,79414.26,NaN,NaN,0.00
4,5003021368,001-519-663 10,142589.38,Учитель,163810.32,Учитель,153199.85,153199.85,153199.85,306399.70
...,...,...,...,...,...,...,...,...,...,...
130956,9715217689,173-260-871 66,NaN,NaN,67358.00,Воспитатель,67358.00,67358.00,NaN,67358.00
130957,9715217689,184-638-768 23,NaN,NaN,142971.65,Педагог-организатор,142971.65,142971.65,NaN,142971.65
130958,9715217689,184-820-648 92,NaN,NaN,79250.00,Специалист по закупкам,79250.00,NaN,NaN,0.00
130959,9715217689,187-933-260 19,NaN,NaN,125727.05,Учитель,125727.05,125727.05,125727.05,125727.05


In [250]:
def load_data(file):
    df = pd.read_excel(path_to_data + file + '.xlsx')
    df.stv = df.apply(lambda row: to_double(row, 'stv'), axis=1)
    sample = df[((df.type == 'Основное место работы') | (df.type == 'Внутреннее совместительство')) & (df.inn != 9718071371)]
    sums = sample.iloc[:, [0,1,2,3]].groupby(['inn', 'snils']).sum().reset_index()
    sums = pd.merge(sums, sample[sample.type == 'Основное место работы'], how='left', on='snils')
    sums = sums[['inn_x', 'snils', 'sum_x', 'stv', 'job']].drop_duplicates(['inn_x', 'snils', 'sum_x'])
    sums = sums.rename(columns = {'inn_x': 'inn', 'sum_x': 'sum_' + file, 'job':'job_' + file, 'stv': 'stv_' + file})
    return sums

In [305]:
prep = pd.merge(load_data('jan'), load_data('feb'), how='outer', on=['inn', 'snils'])
prep['summ'] = prep.apply(lambda row: summ_isp(row, ['jan', 'feb']), axis=1)
val_1_3_ppl = prep[(prep.summ.isna() == False) & (prep.summ > 0)]
val_1_3 = prep[['inn', 'sum_jan', 'sum_feb']].groupby('inn').sum().reset_index()
val_1_3['sum'] = val_1_3.sum_jan + val_1_3.sum_feb
prep = prep[['inn', 'summ']].groupby('inn').sum().reset_index()
val_1_3 = pd.merge(prep[['inn', 'summ']], val_1_3[['inn', 'sum']], how='left', on='inn')
val_1_3['ans'] = val_1_3['summ'] / val_1_3['sum']
val_1_3

,inn,summ,sum,ans
0,5003021368,1.554936e+08,1.811097e+08,0.858560
1,5003021495,1.885417e+08,2.205783e+08,0.854761
2,5003021640,1.383257e+08,1.614748e+08,0.856640
3,5003096290,8.109695e+07,1.030989e+08,0.786594
4,5030032182,4.509598e+07,5.465168e+07,0.825153
...,...,...,...,...
591,7743939315,1.446462e+07,1.809005e+07,0.799590
592,7743940590,5.645479e+07,7.001209e+07,0.806358
593,9705101759,0.000000e+00,9.733310e+07,0.000000
594,9715206976,7.509696e+07,9.860489e+07,0.761595


In [306]:
print_df(val_1_3, 'val_1_3')
print_df(val_1_3_ppl, 'val_1_3_ppl')

In [307]:
val_1_3[val_1_3.ans < 0.75]

,inn,summ,sum,ans
12,7701032028,6862144.44,9803410.62,0.699975
13,7701050524,21026481.30,32631479.83,0.644362
14,7701113894,6076397.07,9258446.06,0.656309
16,7701375995,7877915.88,10951698.87,0.719333
19,7701406844,8955616.69,13207441.28,0.678074
...,...,...,...,...
579,7743085712,48772703.92,70623290.12,0.690604
580,7743085720,30113264.41,47020093.95,0.640434
581,7743085744,44283794.54,67107189.60,0.659896
590,7743937357,32665009.34,54765527.19,0.596452


In [79]:
out = res[['inn', 'szp_ped']].groupby('inn').count().rename(columns={'szp_ped': 'Количество педов'})
out

,Количество педов
inn,
5003021368,433
5003021495,447
5003021640,371
5003096290,285
5030032182,126
...,...
7743939315,42
7743940590,141
9705101759,0


In [92]:
out = df[df.job.isin(['Учитель'])].drop_duplicates(['inn', 'snils'])[['inn', 'snils']].groupby('inn').count().rename(columns={'snils':'Количество педов'})

In [94]:
out
wb = Workbook()
ws = wb.active
for  i in df_to_row(out, header=True):
    ws.append(i)
wb.save('teach.xlsx')

In [ ]:
school_sum = sample[['inn', 'sum']].groupby('inn').sum()

In [ ]:
school_sum

In [ ]:
s = sums.groupby(['snils']).count().reset_index()
wb = Workbook()
ws = wb.active
for  i in df_to_row(s, header=True):
    ws.append(i)
wb.save('test.xlsx')